In [47]:
import pdfplumber
import pandas as pd
import re

def extract_transaction_lines(page):
    """Extract transaction lines with improved detection using regex."""
    text = page.extract_text()
    # Regex pattern to match transaction lines more accurately
    pattern = r'\d{2}-\w{3}-\d{4}\s+[TC]\s+.*?\d+,\d+\.\d{2}[DC][rR]?'
    transactions = re.findall(pattern, text.replace('\n', ' '))
    
    data = []
    for transaction in transactions:
        # Further split each transaction into components
        parts = transaction.split()
        date = parts[0]
        transaction_type = parts[1]
        description = ' '.join(parts[2:-3])
        amount = parts[-3]
        balance = parts[-2] + parts[-1]
        data.append([date, transaction_type, description, amount, balance])
    
    return data

def process_pdf_to_excel(pdf_path, excel_path):
    """Process PDF and export extracted transaction data to Excel, with improvements."""
    with pdfplumber.open(pdf_path) as pdf, pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
        all_transactions = []
        for page in pdf.pages:
            transactions = extract_transaction_lines(page)
            all_transactions.extend(transactions)
        
        if all_transactions:
            df = pd.DataFrame(all_transactions, columns=['Date', 'Type', 'Description', 'Amount', 'Balance'])
            df.to_excel(writer, sheet_name='Transactions', index=False)

if __name__ == "__main__":
    pdf_file_path = 'test3.pdf'
    output_excel_path = 'extracted_transactions_improved.xlsx'
    process_pdf_to_excel(pdf_file_path, output_excel_path)